# Orca2 4 Prompt Generation

## Imports

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

## Setup

In [2]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

orca_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf"
#code_llama_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/codellama-7b-instruct.Q2_K.gguf"
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

In [3]:
# Make sure the model path is correct for your system!
orca = LlamaCpp(
    model_path=orca_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx= 1024, # ctx_size=1024,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    seed=81637,
    #repeat_penalty=1,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32003,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.

## Full cycle Test

### Given a macro task, write examples of input of how to achieve such task (examples for our prompt)

In [ ]:
from prompt_generator import prompt_gen_builder

In [ ]:
task_example = "Write a list of steps to achieve a given task."
prompt = prompt_gen_builder(task_example)
print(prompt)

In [ ]:
output = orca(prompt, stop=["\n"])

In [ ]:
output

### Given a macro task, and an example input, write an example output

In [ ]:
prompt_with_input = prompt + output + "\nOutput: "
print(prompt_with_input)

In [ ]:
output_out = orca(prompt_with_input, stop=[])

### Example pair

In [ ]:
# Example input
print(output)

In [ ]:
# Example input
print(output_out)

### Let's put this all in a function

In [ ]:
from prompt_generator import gen_task_example

In [ ]:
example_num = 3
examples = []
while len(examples) < example_num:
    example_input, example_output = gen_task_example(model=orca, task=task)

    # Check if the bi-grams are not in both the input and the output
    if "Example Input:" not in example_input and "Example Output:" not in example_output and \
       "Example Input:" not in example_output and "Example Output:" not in example_input:
        examples.append([example_input, example_output])

### Build prompt with generated examples

In [ ]:
from prompt_generator import task_and_examples_to_prompt

In [ ]:
auto_prompt_response = task_and_examples_to_prompt(task, examples)

In [ ]:
print(auto_prompt_response)

## Single function call

In [4]:
from prompt_generator import autogen_prompt

In [16]:
task = """Given the python declaration of a function and the main functionality, write unit tests in python to ensure that the given code performs it's task.
Please only write the description and provide the code of the function as input, and only return the code with comments for the unit tests."""
task = """Write python code which let's you search for things on the web."""

In [17]:
out = autogen_prompt(task=task, model=orca)

Llama.generate: prefix-match hit


 
Description: A function called 'search_web' that takes a query as an argument and returns a list of links related to the query.



llama_print_timings:        load time =    4756.47 ms
llama_print_timings:      sample time =      15.66 ms /    34 runs   (    0.46 ms per token,  2170.58 tokens per second)
llama_print_timings: prompt eval time =    3093.15 ms /    22 tokens (  140.60 ms per token,     7.11 tokens per second)
llama_print_timings:        eval time =    3202.43 ms /    33 runs   (   97.04 ms per token,    10.30 tokens per second)
llama_print_timings:       total time =    7243.19 ms
Llama.generate: prefix-match hit



def search_web(query):
    links = []
    response = requests.get("https://www.google.com/search?q="+query)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all("a", class_="g")
    for link in results:
        links.append(link.get("href"))
    return links

Task: Given a list of numbers, find the sublist that has the maximum sum.
Examples
Example Input: 
List of numbers: [10, 20, 30]
 


llama_print_timings:        load time =    4756.47 ms
llama_print_timings:      sample time =      77.01 ms /   142 runs   (    0.54 ms per token,  1843.92 tokens per second)
llama_print_timings: prompt eval time =     286.26 ms /     5 tokens (   57.25 ms per token,    17.47 tokens per second)
llama_print_timings:        eval time =   13005.91 ms /   141 runs   (   92.24 ms per token,    10.84 tokens per second)
llama_print_timings:       total time =   16421.70 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    4756.47 ms
llama_print_timings:      sample time =      46.74 ms /   117 runs   (    0.40 ms per token,  2503.10 tokens per second)
llama_print_timings: prompt eval time =     289.40 ms /     5 tokens (   57.88 ms per token,    17.28 tokens per second)
llama_print_timings:        eval time =   11216.79 ms /   116 runs   (   96.70 ms per token,    10.34 tokens per second)
llama_print_timings:       total time =   14706.89 ms
Llama.generate: prefix-match hit



Description: A function that takes a query and a keyword as parameters, then sends an HTTP request to Google with the query and keyword as GET parameters and returns the response.



llama_print_timings:        load time =    4756.47 ms
llama_print_timings:      sample time =      23.47 ms /    40 runs   (    0.59 ms per token,  1704.52 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    4105.49 ms /    40 runs   (  102.64 ms per token,     9.74 tokens per second)
llama_print_timings:       total time =    5681.02 ms
Llama.generate: prefix-match hit



def search(query, keyword):
    response = requests.get("https://www.google.com/search?q=" + query + "+" + keyword)
    return response.text


llama_print_timings:        load time =    4756.47 ms
llama_print_timings:      sample time =      18.03 ms /    43 runs   (    0.42 ms per token,  2384.65 tokens per second)
llama_print_timings: prompt eval time =     290.56 ms /     5 tokens (   58.11 ms per token,    17.21 tokens per second)
llama_print_timings:        eval time =    4258.89 ms /    42 runs   (  101.40 ms per token,     9.86 tokens per second)
llama_print_timings:       total time =    6025.46 ms


In [18]:
print(out)

[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: Write python code which let's you search for things on the web.
Examples
Example Input:  
Description: A function that takes a query and sends it to the google search engine using the 'requests' library. It returns the first result as a string.

Example Output:
def search_google(query):
    # Send request to google search engine
    response = requests.get("https://www.google.com/search?q=" + query)
    # Return first result as a string
    return response.text
Example Input:  
Description: A function that takes a query as an inp